# 뉴스 크롤링

In [1]:
pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.3 MB/s eta 0:00:00


In [2]:
pip install webdriver-manager

In [ ]:
!apt-get update
!apt install chromium-chromedriver

In [81]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)

In [82]:
# 기사 링크 크롤링
# 카테고리당 100개 = 5페이지

url_list = []   # url 저장 리스트
a_list = []     # a 태그 저장 리스트

for category in range(3, 4):     # 6
    for page in range(1, 2):  # 1, 6
        url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={100 + category}#&date=%2000:00:00&page={page}'
        browser.get(url)

        time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")
        a_list.extend(soup.select(".type06_headline dt+dt a"))
        a_list.extend(soup.select(".type06 dt+dt a"))

        print("카테고리 : ", 100 + category, "페이지 : ", page)

for a in a_list:  # 링크
    url_list.append(a["href"])

카테고리 :  103 페이지 :  1


In [ ]:
url_list

In [93]:
# 수집한 링크의 제목, 본문, 날짜 크롤링

title = []
content = []
date = []

for url in url_list:
      browser.get(url)

      time.sleep(1)

      soup = BeautifulSoup(browser.page_source, "html.parser")

      title.extend(soup.select("#title_area span"))       # 제목

      c = soup.find_all(attrs={"id" : "dic_area"})        # 본문

      while c[0].find(attrs={"class" : "end_photo_org"}):         # 이미지 있는 만큼
        c[0].find(attrs={"class" : "end_photo_org"}).decompose()  # 본문 이미지에 있는 글자 없애기

      while c[0].find(attrs={"class" : "vod_player_wrap"}):       # 영상 있는 만큼
        c[0].find(attrs={"class" : "vod_player_wrap"}).decompose()  # 본문 영상에 있는 글자 없애기

      content.extend(c)

      date.extend(soup.select("._ARTICLE_DATE_TIME"))     # 날짜

In [94]:
# 위에서 수집한 태그 속에서 내용만 뽑아서 담는다.

def clean(article):   # 전처리
  article = re.sub('\n','',article)
  article = re.sub('\t','',article)
  article = re.sub('\u200b','',article)
  article = re.sub('\xa0','',article)
  article = re.sub('([a-zA-Z])','',article)
  article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)
  article = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',article)

  return article

title_list = []     # 제목
content_list = []   # 본문
date_list = []      # 날짜

for t in title:
  title_list.append(t.text)

for c in content:
  content_list.append(clean(c.text))

for d in date:
  date_list.append(d["data-date-time"])

In [ ]:
# 데이터 프레임으로 변환

article_df = pd.DataFrame({"날짜" : date_list,
                           "제목" : title_list,
                           "본문" : content_list,
                           "링크" : url_list})

article_df

# 전처리

**KoNLPy**
- 한국어 정보처리를 위한 파이썬 패키지
- https://konlpy.org/ko/latest/api/konlpy.tag/#okt-class

**CountVectorizer**
- 카운트 기반 벡터화
- https://bkshin.tistory.com/entry/NLP-6-%EC%B9%B4%EC%9A%B4%ED%84%B0-%EA%B8%B0%EB%B0%98-%EB%B2%A1%ED%84%B0%ED%99%94CountVectorizer%EC%99%80-TF-IDF-%EB%B2%A1%ED%84%B0%ED%99%94

In [10]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 48.6 MB/s eta 0:00:00


In [14]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer

In [105]:
okt = Okt()
nouns_list = []

for content in article_df["본문"]:
  nouns_tmp = []

  content = clean(content)         # 전처리
  content = okt.nouns(content)     # 명사 추출

  vector = CountVectorizer().fit(content)
  print(vector.vocabulary_)


{'한국': 146, '마약': 33, '통제': 133, '사회': 55, '우리나라': 80, '드럭': 31, '인덱스': 94, '기록': 18, '인구': 93, '적발': 100, '사범': 51, '수로': 63, '이상': 92, '이면': 91, '진입': 116, '의미': 87, '유혹': 83, '곳곳': 10, '사리': 49, '가운데': 1, '사이': 53, '노출': 24, '퐁당': 135, '이란': 90, '정도': 103, '몰래': 37, '타인': 131, '약물': 71, '중독': 112, '사례': 48, '종종': 109, '대해': 29, '다른': 26, '사람': 47, '음료': 85, '술잔': 65, '성분': 59, '일명': 97, '성범죄': 58, '금전': 17, '갈취': 5, '범죄': 42, '가능성': 0, '최근': 122, '강남': 6, '학원': 145, '일대': 96, '학교': 142, '교문': 13, '필로폰': 140, '엑스터시': 74, '학생': 144, '일당': 95, '충격': 125, '역시': 77, '학부모': 143, '요구': 79, '지난해': 114, '유흥주점': 84, '종업원': 108, '손님': 61, '추정': 124, '과다복용': 12, '사망': 50, '사건': 46, '발생': 41, '동호회': 30, '만난': 34, '여성': 75, '남성': 22, '경찰': 9, '위험': 81, '확인': 153, '전자담배': 101, '경우': 8, '지난': 113, '플루': 138, '자신': 99, '모금': 35, '정신': 104, '신고': 69, '해당': 149, '긴급': 19, '체포': 121, '진술': 115, '대마': 27, '젤리': 106, '미국': 39, '부지역': 43, '캐나다': 128, '제품': 105, '합법': 147, '해외여행': 151, '대표': 28, '하리보': 141